Data Science Capstone Week 4



Import packages

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup
from urllib.request import urlopen

Data Preparation - New York

In [2]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json --output 'newyork_data.json'

with open('newyork_data.json', 'r') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
neighborhoods = pd.DataFrame(columns=['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] )

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighbourhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  113k  100  113k    0     0  59585      0  0:00:01  0:00:01 --:--:-- 59554


In [3]:
neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Data Preparation - Toronto

In [4]:
pc_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pc_html = urlopen(pc_url)
pc_bs = BeautifulSoup(pc_html, 'html.parser')
pc_tbl = pc_bs.find_all('table')
pc_tbl2 = pc_tbl[0]

header = [i.text.strip() for i in pc_tbl2.find_all('th')]
cell = pc_tbl2.find_all('td')

pc_df = pd.DataFrame(columns=header)

for i in range(0, int(len(cell)/3)):
    row = [j.text.strip() for j in cell[3*i:3*i+3]]
    if row[1] != "Not assigned":
        if ',' in row[2]:
            for nei in row[2].split(','):
                pc_df = pc_df.append({header[0]:row[0],header[1]:row[1],header[2]:nei.strip()},
                                 ignore_index=True)
        else:
            pc_df = pc_df.append({header[0]:row[0],header[1]:row[1],header[2]:row[2]},
                                 ignore_index=True)

def toronto_geocoder(row):
    geolocator = Nominatim(user_agent="ny_explorer")
    print(row["Neighbourhood"])
    try:
        location = geolocator.geocode(row["Neighbourhood"] + ", Toronto, Canada")
        row["Latitude"] = location.latitude
        row["Longitude"] = location.longitude
    finally:
        return row

geo_df = pd.read_csv("https://cocl.us/Geospatial_data")
pc_df2 = pc_df.merge(geo_df, on="Postal Code")
pc_df2 = pc_df2.apply(toronto_geocoder, axis=1)

Parkwoods
Victoria Village
Regent Park
Harbourfront
Lawrence Manor
Lawrence Heights
Queen's Park
Ontario Provincial Government
Islington Avenue
Humber Valley Village
Malvern
Rouge
Don Mills
Parkview Hill
Woodbine Gardens
Garden District
Ryerson
Glencairn
West Deane Park
Princess Gardens
Martin Grove
Islington
Cloverdale
Rouge Hill
Port Union
Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
Bloordale Gardens
Old Burnhamthorpe
Markland Wood
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Wilson Heights
Downsview North
Thorncliffe Park
Richmond
Adelaide
King
Dufferin
Dovercourt Village
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Broadview North (Old East York)
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
Kennedy Park
Ionview
East Birchmount Park
Bayview Village
Downsvi

In [5]:
pc_df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,M6A,North York,Lawrence Manor,43.722079,-79.437507


In [6]:
CLIENT_ID = 'KSDRW32BYILNOSUYGIWMAQQU0RMTC105NSWYBDEVV0R2XEWF' # your Foursquare ID
CLIENT_SECRET = 'UQKFFWIF5LMW5LAWGBO1XIAK2KYHJZ5VQA3F340FWYPOJFPJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
ny_nearby = getNearbyVenues(neighborhoods["Neighbourhood"], neighborhoods["Latitude"], neighborhoods["Longitude"], radius=500)
tr_nearby = getNearbyVenues(pc_df2["Neighbourhood"], pc_df2["Latitude"], pc_df2["Longitude"], radius=500)

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [9]:
ny_nearby.shape

(10047, 7)

In [10]:
tr_nearby.shape

(6212, 7)

In [11]:
ny_nearby.groupby('Neighborhood').count()["Venue"]

Neighborhood
Allerton          28
Annadale          12
Arden Heights      5
Arlington          5
Arrochar          19
                ... 
Woodhaven         25
Woodlawn          26
Woodrow           17
Woodside          79
Yorkville        100
Name: Venue, Length: 302, dtype: int64

In [12]:
tr_nearby.groupby('Neighborhood').count()["Venue"]

Neighborhood
Adelaide            100
Agincourt            12
Agincourt North      28
Albion Gardens        7
Alderwood             6
                   ... 
Woodbine Heights      8
York Mills           16
York Mills West      16
York University      18
Yorkville           100
Name: Venue, Length: 208, dtype: int64

In [40]:
def neibourhood_score(df):
    pos_list = ["Light Rail Station", "Metro Station", "Tram Station", "Train Station",
                "Bus Stop", "Bus Station", "Police Station", "Fire Station", "Hospital",
                "Medical Center", "Pier", "Shopping Plaza", "Shopping Mall", "Supermarket",
               "Grocery Store", "Department Store", "Convenience Store", "Waterfront", "Park",
               "National Park","Harbor / Marina"]
    neg_list = ["Prison", "Military Base", "Power Plant", "Waste Facility", "Airport",
                "Gas Station", "Funeral Home"]
    onehot = pd.get_dummies(df[['Venue Category']], prefix="", prefix_sep="")
    onehot['Neighborhood'] = df['Neighborhood']
    onehot_grp = onehot.groupby("Neighborhood").sum()
    onehot_col = onehot_grp.columns
    score = onehot_grp.get(set(onehot_col).intersection(set(pos_list))).sum(axis=1) - onehot_grp.get(set(onehot_col).intersection(set(neg_list))).sum(axis=1)
    return score

In [41]:
ny_score = neibourhood_score(ny_nearby)
ny_score.sort_values()

Neighborhood
Bellerose              -1
Broadway Junction      -1
Bath Beach             -1
Flatbush                0
Greenwich Village       0
                       ..
Woodside                8
Battery Park City       9
Kingsbridge Heights     9
Sunnyside Gardens      10
West Farms             11
Length: 302, dtype: int64